In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MediWatch_project\\research'

In [3]:
# Change present working directory to root directory
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Dheer\\OneDrive\\Desktop\\MLOps\\MediWatch_project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [6]:
from Mediwatch_project.constants import *
from Mediwatch_project.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [8]:
import os
import pandas as pd
import urllib.request as request
import zipfile
from Mediwatch_project import logger
from Mediwatch_project.utils.common import get_size


In [10]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

In [11]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-08-04 20:02:46,071: INFO: common: Raw YAML content from WindowsPath('config/config.yaml'): {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://raw.githubusercontent.com/dheeraj24797/MediWatch_project/main/diabetic_data.zip', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}} (type=<class 'dict'>)]
[2025-08-04 20:02:46,079: INFO: common: YAML file: config\config.yaml loaded successfully]
[2025-08-04 20:02:46,083: INFO: common: Raw YAML content from WindowsPath('params.yaml'): {'model_params': {'learning_rate': 0.01, 'n_estimators': 100}} (type=<class 'dict'>)]
[2025-08-04 20:02:46,087: INFO: common: YAML file: params.yaml loaded successfully]
[2025-08-04 20:02:46,091: INFO: common: Raw YAML content from WindowsPath('schema.yaml'): {'COLUMNS': 'encounter_id:int64 patient_nbr:int64 race:object gender:object age:object weight:object admission_type_id:int64 discharge_dispositio